In [1]:
import re
import socket
from elasticsearch import Elasticsearch
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from operator import itemgetter
import nltk
import nltk.data
import requests
import os
import sys
import pandas as pd

sys.path.append('../../')

import globals
from qa_data import QAPair
import bioasq_eval

Home path : /opt/bioasq/col-un-bioasq11
Eval path : /opt/bioasq/Evaluation-Measures


In [2]:
def rank_document(question, doc_id, doc_title, doc_abstract):
    passages_ranked = []
    chunks_title = ranking.split_chunks(doc_title)
    title_passages_ranked = [ rank_bioasq_passage(doc_id, question, chunk, 'title') for chunk in chunks_title ]
    chunks_abstract = ranking.split_chunks(doc_abstract)
    abstract_passages_ranked = [ rank_bioasq_passage(doc_id, question, chunk, 'abstract') for chunk in chunks_abstract ]
    return title_passages_ranked + abstract_passages_ranked

In [5]:
def extract_rank_answer_candidates(question, docs):
    snippets = []
    w = np.linspace(0,0.0,len(docs))[::-1]
    count_pass1 = 0
    for i, doc in enumerate(docs):
        doc_id = doc.replace(globals.BIOASQ.doc_relative_url,'')
        doc_id, doc_title, doc_abstract = bioasq_util.get_doc(doc_id, index_name)
        #doc_id = doc[0].replace(doc_relative_url,'')
        #doc_title = doc[1]
        #doc_abstract = doc[2]
        snippets_ranked = rank_document(question, doc_id, doc_title, doc_abstract)
        count_pass1 += len(snippets_ranked)
        snippets_ranked = [ s for s in  snippets_ranked if s['score'] >= 0.6 ]
        for s in snippets_ranked:
            s['score'] = s['score'] + w[i]
        snippets.extend(snippets_ranked)
    #print('Percentage of remaining passages {}'.format(len(snippets)/count_pass1))
    return snippets

In [6]:
import json
from tqdm import tqdm
import numpy as np
import bioasq_util

gs_google_docs = globals.PATH.eval_home + '/examples/aueb_google_docs/aueb_nlp-bioasq6b-submissions/'
index_name = globals.BIOASQ.index

#evaluate over aueb documents
test_batch_docs = [ #('','8b5_ES_30_full.json')
                ('6B1_golden.json', gs_google_docs+'1-aueb-nlp-4.json'),
                ('6B2_golden.json', gs_google_docs+'2-aueb-nlp-4.json'),
                ('6B3_golden.json', gs_google_docs+'3-aueb-nlp-4.json'),
                ('6B4_golden.json', gs_google_docs+'4-aueb-nlp-4.json'),
                ('6B5_golden.json', gs_google_docs+'5-aueb-nlp-4.json')
               ]
    
df = pd.DataFrame(columns=('batch', 'Mean precision', 'Recall', 'F-Measure', 'MAP', 'GMAP'))

for i, batch_file in enumerate(test_batch_docs):
    test_batch_json = json.load(open(batch_file[1]))
    for sample in tqdm(test_batch_json['questions'], position=0):
        snippets = extract_rank_answer_candidates(sample['body'], sample['documents'])
        snippets_sorted = sorted(snippets, key = lambda i: (i['score']), reverse=True)
        sample['snippets'] = snippets_sorted
        sample['documents'] = [ d[0] for d in sample['documents'] ][0:10]
        sample['documents'] = sample['documents'][0:10]
    submission = test_batch_json.copy()
    for q in submission['questions']:
        for s in q['snippets']:
            del s['score']
    submission_file_name =  working_folder + "/" + model_id + '_'+batch_file[1].split('/')[-1]
    json.dump(submission, open(submission_file_name, 'w'))
    docs_score, pass_score = bioasq_eval.get_scores_phaseA(batch_file[0], submission, path_home=eval_home)
    print('Document Scores',docs_score)
    print('Passage Scores',pass_score)
    df.loc[i] = [ batch_file[0].split('.')[0] + '_' + batch_file[1].split('/')[-1].split('.')[0] ] + pass_score

df.to_csv(working_folder + "/" + model_id+'.csv', index=False)

  0%|                                                                                                                                                                                                                    | 0/100 [00:00<?, ?it/s]/home/andresr/.local/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
  0%|                                                                                                                                                                                                                    | 0/100 [00:00<?, ?it/s]

('Error in query: ', '9507713')
'title'


ValueError: too many values to unpack (expected 3)